In [ ]:
!pip install selenium
!pip install scikit-learn
!pip install underthesea

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random
from urllib.parse import urljoin
# !pip install underthesea scikit-learn requests pandas numpy beautifulsoup4 lxml # Uncomment this in Colab/Jupyter if needed
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import numpy as np
import os
import json # ### FIX ###: Import json library
# Thêm thư viện cho parallel crawl (nếu dùng)
from concurrent.futures import ThreadPoolExecutor, as_completed
import traceback # For detailed error logging

print("Block 1: Imports, Cấu hình Nâng cao và Hàm Tiện ích - Đang chạy...")

# --- 1. Cấu hình Nâng cao ---
BASE_HOST = "https://buudien.vn"
CSV_FILENAME = "buudien_ocop_products_detailed_v2_rerun.csv"
BASE_URL_TEMPLATE = "https://buudien.vn/home/Search/index.html?keyword=OCOP&page={page}"

# Trọng số Feature
NAME_WEIGHT = 7

# Tham số TF-IDF Tối ưu
TFIDF_NGRAM_RANGE = (1, 2)
TFIDF_MIN_DF = 2
TFIDF_MAX_DF = 0.8
TFIDF_MAX_FEATURES = 5000
TFIDF_SUBLINEAR_TF = True
TFIDF_NORM = 'l2'

# Số lượng gợi ý
TOP_N_RAW_RECS = 30 # Used for precomputing raw recommendations
TOP_N_FINAL_RECS = 10 # Used for final display/testing

# Tên file lưu trữ kết quả
COSINE_SIM_MATRIX_FILE = 'cosine_similarity_matrix_v2_adv.npy'
INDICES_MAP_FILE = 'product_indices_map_v2_adv.pkl'
PRODUCT_MAP_JSON_FILE = 'product_id_name_map_v2_adv.json' # ### FIX ###: Use JSON extension explicitly
PRECOMPUTED_RECS_JSON_FILE = 'precomputed_recommendations_v2_raw_adv.json' # ### FIX ###: Use JSON extension

# Cấu hình crawl
REQUEST_TIMEOUT = 30
RETRY_ATTEMPTS = 3
SLEEP_MIN = 0.5
SLEEP_MAX = 1.5
CRAWL_MAX_PAGES = 250 # Giới hạn để test nhanh hơn, tăng lên nếu cần crawl hết (e.g., 500 or more)
CRAWL_EMPTY_STOP = 5
MAX_CRAWL_WORKERS = 8

# --- Cấu hình Tiền xử lý Nâng cao ---
STOP_WORDS_FILE = 'vietnamese_stopwords.txt'
# Tải file stop words nếu chưa có (ví dụ cho Colab)
if not os.path.exists(STOP_WORDS_FILE):
    print(f"Đang tải file {STOP_WORDS_FILE}...")
    try:
        stopwords_url = "https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt" # Example source
        stopwords_resp = requests.get(stopwords_url)
        stopwords_resp.raise_for_status()
        with open(STOP_WORDS_FILE, 'w', encoding='utf-8') as f_sw:
            f_sw.write(stopwords_resp.text)
        print("Tải stop words thành công.")
    except Exception as download_err:
        print(f"Lỗi tải stop words: {download_err}. Sử dụng set rỗng.")


VIETNAMESE_STOP_WORDS = set()
if os.path.exists(STOP_WORDS_FILE):
    try:
        with open(STOP_WORDS_FILE, 'r', encoding='utf-8') as f:
            VIETNAMESE_STOP_WORDS = set(line.strip() for line in f if line.strip())
        print(f"Đã tải {len(VIETNAMESE_STOP_WORDS)} stop words.")
    except Exception as e:
        print(f"Lỗi khi tải stop words từ file cục bộ: {e}. Sử dụng set rỗng.")
else:
    print(f"Không tìm thấy file stop words: {STOP_WORDS_FILE}. Sử dụng set rỗng.")

# Từ điển Đồng nghĩa (Giữ nguyên ví dụ của bạn)
SYNONYM_DICT = {
    # Tên Thương hiệu / Nhà sản xuất (Ví dụ)
    'huongfarm': 'huongfarm', 'lạc lạc plus': 'laclacplus', 'phủ quỳ': 'phuquy',
    'xuân anh': 'xuananh', 'đất ngọc': 'datngoc', 'gieo...đặc sản đà lạt': 'gieodacsandl',
    'dương anh 568': 'duonganh568', 'viện nông nghiệp thanh hoá': 'viennnthanhhoa',
    'hải đăng': 'haidang', 'bà ba': 'baba', 'quý thu': 'quythu', 'bà hùng': 'bahung',
    'kim huệ': 'kimhue', 'hữu châu': 'huuchau', 'minh vạn': 'minhvan',
    # ... (Thêm các từ đồng nghĩa khác nếu cần) ...
    'ngũ cốc dinh dưỡng': 'ngucocdinhduong', 'ngũ cốc siêu dinh dưỡng': 'ngucocdinhduong',
    'mì chùm ngây': 'michumngay', 'mì cà rốt': 'micarot', 'mì củ dền': 'micuden',
    'bột sắn dây': 'botsanday', 'tinh bột sắn dây': 'botsanday', 'trà túi lọc': 'tratuiloc',
    'trà xạ đen': 'traxaden', 'chè xanh': 'chexanh', 'trà sơn mật': 'trasonmat',
    'hồng sâm': 'hongsam', 'trà hoa vàng': 'trahoavang', 'chè hảo đạt': 'chehaodat',
    'tôm nõn': 'tomnon', 'trà hoàng thảo mộc': 'trahoangthaomoc', 'cà phê hòa tan': 'caphehoatan',
    'mộng dừa': 'mongdua', 'đẳng sâm': 'dangsam', 'ngọc linh': 'ngoclinh', 'cà phê đăk hà': 'caphedakha',
    'trà giải độc gan': 'tragiaidocgan', 'cà gai leo': 'cagaileo', 'xạ đen': 'xaden',
    'tinh bột nghệ': 'tinhbotnghe', 'viên nghệ mật ong': 'viennghematong', 'nem chua': 'nemchua',
    'nem nướng': 'nemnuong', 'mắm ruốc': 'mamruoc', 'mắm tôm': 'mamtom', 'mắm tép': 'mamtep',
    'nước mắm': 'nuocmam', 'cá cơm': 'cacom', 'khô cá': 'khoca', 'cá lóc': 'caloc',
    'cá sặc rằn': 'casacran', 'cá kèo': 'cakeo', 'cá bống': 'cabong', 'cá thu': 'cathu',
    'cá mòi': 'camoi', 'cá nhệch': 'canhech', 'cá trắm': 'catram', 'bún gạo khô': 'bungaokho',
    'bún khô': 'bunkho', 'hủ tiếu khô': 'hutieukho', 'miến gạo': 'miengao', 'miến dong': 'miendong',
    'gạo lứt': 'gaolut', 'gạo tím than': 'gaotimthan', 'nếp cẩm': 'nepcam',
    'đông trùng hạ thảo': 'dongtrunghathao', 'hạt mắc ca': 'macca', 'hạt macca': 'macca',
    'macadamia': 'macca', 'hạt điều': 'hatdieu', 'rang muối': 'rangmuoi', 'bánh đa nem': 'banhdanem',
    'bánh đa vừng': 'banhdavung', 'bánh tráng': 'banhtrang', 'cơm cháy': 'comchay',
    'chà bông': 'chabong', 'sữa chua': 'suachua', 'yến sào': 'yensao', 'tổ yến': 'toyen',
    'tinh dầu': 'tinhdau', 'sả chanh': 'sachanh', 'húng chanh': 'hungchanh', 'tía tô': 'tiato',
    'sấy dẻo': 'saydeo', 'sấy khô': 'saykho', 'sấy giòn': 'saygion', 'sấy thăng hoa': 'saythanghoa',
    'sấy truyền thống': 'saytruyenthong', 'ngâm muối': 'ngammuoi', 'muối chua': 'muoichua',
    'hút chân không': 'hutchankhong', 'nguyên chất': 'nguyenchat', 'cao cấp': 'caocap',
    'thượng hạng': 'thuonghang', 'hữu cơ': 'huuco', 'organic': 'huuco', 'túi lọc': 'tuiloc',
    'dạng bột': 'dangbot', 'dạng viên': 'dangvien',
    'lâm đồng': 'lamdong', 'đà lạt': 'dalat', 'bến tre': 'bentre', 'tiền giang': 'tiengiang',
    'hà giang': 'hagiang', 'cao bằng': 'caobang', 'bắc kạn': 'backan', 'thái nguyên': 'thainguyen',
    'ninh bình': 'ninhbinh', 'thanh hóa': 'thanhhoa', 'nghệ an': 'nghean', 'hà tĩnh': 'hatinh',
    'quảng bình': 'quangbinh', 'quảng trị': 'quangtri', 'thừa thiên huế': 'hue',
    'quảng nam': 'quangnam', 'quảng ngãi': 'quangngai', 'bình định': 'binhdinh', 'phú yên': 'phuyen',
    'khánh hòa': 'khanhhoa', 'ninh thuận': 'ninhthuan', 'bình thuận': 'binhthuan', 'kon tum': 'kontum',
    'gia lai': 'gialai', 'đắk lắk': 'daklak', 'đắc lắc': 'daklak', 'đắk nông': 'daknong',
    'đắc nông': 'daknong', 'bình phước': 'binhphuoc', 'bình dương': 'binhduong', 'tây ninh': 'tayninh',
    'đồng nai': 'dongnai', 'bà rịa vũng tàu': 'vungtau', 'long an': 'longan', 'đồng tháp': 'dongthap',
    'an giang': 'angiang', 'cần thơ': 'cantho', 'vĩnh long': 'vinhlong', 'hậu giang': 'haugiang',
    'sóc trăng': 'soctrang', 'bạc liêu': 'baclieu', 'cà mau': 'camau', 'kiên giang': 'kiengiang',
    'phú quốc': 'phuquoc',
}

TOKEN_MIN_LEN = 2
TOKEN_MAX_LEN = 20

# --- 2. Hàm Tiện ích ---
session = requests.Session()
# Cập nhật User-Agent để tránh bị chặn
session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})

def apply_synonyms(text):
    """Áp dụng thay thế từ đồng nghĩa."""
    if not isinstance(text, str): return ""
    processed_text = text.lower() # Chuyển lowercase trước khi thay thế
    for k, v in SYNONYM_DICT.items():
        # Thay thế bằng regex để đảm bảo là từ riêng biệt (\b)
        processed_text = re.sub(r'\b{}\b'.format(re.escape(k)), v, processed_text, flags=re.IGNORECASE)
    return processed_text

def advanced_tokenizer(text):
    """Tokenizer nâng cao: synonyms, word_tokenize, stop words, length filter."""
    if not isinstance(text, str): return []

    # 1. Áp dụng synonyms (đã chuyển lowercase trong apply_synonyms)
    text = apply_synonyms(text)

    # 2. Chuẩn hóa cơ bản (bỏ ký tự đặc biệt, số) - Giữ lại khoảng trắng và dấu gạch dưới (do underthesea)
    text = re.sub(r'[^\w\s_]', '', text, flags=re.UNICODE) # Giữ gạch dưới cho từ ghép underthesea
    text = re.sub(r'\d+', '', text)

    # 3. Tokenize bằng underthesea
    try:
        # Đặt trong try-except vì underthesea có thể lỗi với input lạ
        tokenized_text = word_tokenize(text, format="text")
        tokens = tokenized_text.split()
    except Exception as tokenize_err:
        # print(f"Lỗi underthesea tokenizer cho text: '{text[:100]}...' - {tokenize_err}")
        tokens = text.split() # Fallback về split theo khoảng trắng

    # 4. Lọc Stop words và giới hạn độ dài
    filtered_tokens = [
        token for token in tokens
        if token not in VIETNAMESE_STOP_WORDS and TOKEN_MIN_LEN <= len(token) <= TOKEN_MAX_LEN
    ]

    return filtered_tokens # Trả về list các token đã xử lý

def safe_int_convert(value):
    """Chuyển đổi an toàn sang integer, trả về None nếu lỗi."""
    try:
        # Xử lý trường hợp float trước khi chuyển int (ví dụ: 4.0)
        return int(float(value))
    except (ValueError, TypeError, OverflowError):
        return None

def clean_price(price_text):
    """Làm sạch và chuyển đổi giá sang integer."""
    if not isinstance(price_text, str): return 0
    # Bỏ "đ", ".", ",", khoảng trắng và các ký tự không phải số
    price_text = price_text.lower().replace('đ', '').replace('.', '').replace(',', '').strip()
    price_cleaned = re.sub(r'[^\d]', '', price_text)
    price_int = safe_int_convert(price_cleaned)
    return price_int if price_int is not None else 0 # Trả về 0 nếu không chuyển đổi được

print("Block 1: Hoàn tất.")

In [ ]:
print("Block 2: Định nghĩa các Hàm Crawl Dữ liệu (Phiên bản V2 - Fix Pagination) - Đang chạy...")

# --- Hàm Crawl Chi tiết Sản phẩm (Phiên bản V2 - Fix Lỗi 'match') ---
def get_product_details_v2(product_url, existing_data):
    """Hàm crawl chi tiết sản phẩm, cố gắng lấy description, origin, producer, ocop_rating."""
    details = {"description": "", "origin": "", "ocop_rating": None, "producer": ""}
    for attempt in range(RETRY_ATTEMPTS):
        try:
            # In log truy cập (có thể comment bớt nếu quá nhiều)
            # print(f"   - Đang truy cập chi tiết (v2): {product_url}")
            time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX)) # Giảm sleep khi dùng parallel
            response = session.get(product_url, timeout=REQUEST_TIMEOUT)
            response.raise_for_status() # Kiểm tra lỗi HTTP (4xx, 5xx)
            soup = BeautifulSoup(response.text, 'lxml') # Sử dụng lxml cho parser mạnh mẽ hơn

            # Lấy Description
            desc_element = soup.select_one('div.wp_content_tab_description_product')
            details["description"] = ' '.join(desc_element.stripped_strings) if desc_element else ""

            # Lấy Origin (Xuất xứ) - Ưu tiên từ thông tin giao hàng, sau đó đến bảng chi tiết
            origin_text = ""
            origin_gui_tu = soup.select_one('div.kv_store_info div.drop_kv_giaohang') # Selector có thể cần cập nhật
            if origin_gui_tu and origin_gui_tu.text.strip():
                 origin_text = ' '.join(origin_gui_tu.stripped_strings)

            detail_table = soup.select_one('table.tb_parameter_product') # Selector cho bảng chi tiết
            if detail_table and not origin_text: # Chỉ tìm trong bảng nếu chưa có từ nguồn khác
                 for row in detail_table.find_all('tr'):
                     cells = row.find_all('td')
                     if len(cells) == 2:
                         label = cells[0].text.strip().lower() # Chuyển label sang lowercase để so sánh dễ hơn
                         value = cells[1].text.strip()
                         if label and value and label in ["xuất xứ", "nơi sản xuất", "tỉnh thành"]: # So sánh lowercase
                             origin_text = value
                             break # Thoát khi tìm thấy
            details["origin"] = origin_text.strip()

            # Lấy Producer (Nhà sản xuất/Thương hiệu/Gian hàng) - Ưu tiên tên shop, sau đó đến bảng
            producer_text = ""
            shop_name_element = soup.select_one('div.info_store h3 a') # Lấy link trong h3 nếu có
            if shop_name_element and shop_name_element.text.strip():
                producer_text = shop_name_element.text.strip()
            elif shop_name_element is None: # Nếu không có link, thử lấy text của h3
                 shop_name_element = soup.select_one('div.info_store h3')
                 if shop_name_element and shop_name_element.text.strip():
                     producer_text = shop_name_element.text.strip()

            if detail_table and not producer_text: # Chỉ tìm trong bảng nếu chưa có
                 for row in detail_table.find_all('tr'):
                     cells = row.find_all('td')
                     if len(cells) == 2:
                         label = cells[0].text.strip().lower()
                         value = cells[1].text.strip()
                         if label and value and label in ["thương hiệu", "nhà sản xuất", "gian hàng"]:
                             producer_text = value
                             break
            details["producer"] = producer_text.strip()

            # Lấy OCOP Rating - Ưu tiên từ list page, sau đó tìm trong description, cuối cùng là bảng
            ocop_rating = existing_data.get("ocop_rating_from_list") # Lấy từ list trước (đã là int hoặc None)
            match = None # Initialize match to None

            if ocop_rating is None: # Chỉ tìm nếu chưa có từ list
                if details["description"]:
                    # Tìm trong description trước (regex tìm số sau 'OCOP'/'Hạng' và trước 'sao')
                    match = re.search(r'(?:OCOP|Hạng)\s*[:\s-]*\s*(\d)\s*sao', details["description"], re.IGNORECASE)
                    if match: # Kiểm tra ngay sau khi gán
                        ocop_rating = safe_int_convert(match.group(1))

                # Nếu vẫn chưa có, tìm trong bảng chi tiết
                if detail_table and ocop_rating is None:
                    for row in detail_table.find_all('tr'):
                         cells = row.find_all('td')
                         if len(cells) == 2:
                             label = cells[0].text.strip().lower() # Lowercase label
                             value = cells[1].text.strip()
                             if label and value and label in ["chứng nhận ocop", "hạng sao ocop", "ocop"]:
                                  # Tìm số đầu tiên trong value
                                  match_table = re.search(r'(\d)', value)
                                  if match_table:
                                      ocop_rating = safe_int_convert(match_table.group(1))
                                      break # Thoát vòng lặp khi tìm thấy

            details["ocop_rating"] = ocop_rating # Gán kết quả cuối cùng (có thể vẫn là None)
            return details # Trả về kết quả nếu thành công

        except requests.exceptions.RequestException as e:
            print(f"   - Lỗi mạng/HTTP chi tiết (lần {attempt+1}/{RETRY_ATTEMPTS}) URL: {product_url} - Lỗi: {e}")
        except AttributeError as ae: # Lỗi thường gặp khi selector không tìm thấy element
             print(f"   - Lỗi AttributeError chi tiết (lần {attempt+1}/{RETRY_ATTEMPTS}) URL: {product_url} - Lỗi: {ae} (Kiểm tra selector)")
        except Exception as e:
            print(f"   - Lỗi khác khi crawl chi tiết (lần {attempt+1}/{RETRY_ATTEMPTS}) URL: {product_url} - Lỗi: {type(e).__name__} - {e}")
            # traceback.print_exc() # Uncomment để xem full traceback nếu cần debug sâu

        if attempt == RETRY_ATTEMPTS - 1:
            print(f"   - !!! Bỏ qua chi tiết sau {RETRY_ATTEMPTS} lần thử: {product_url}")
            # Trả về dictionary với giá trị mặc định hoặc lỗi
            details["description"] = f"Lỗi crawl chi tiết sau {RETRY_ATTEMPTS} lần thử."
            return details
        time.sleep(random.uniform(2, 4) * (attempt + 1)) # Backoff tăng dần giữa các lần thử

    print(f"   - !!! Lỗi không xác định sau vòng lặp retry cho URL: {product_url}")
    return details


# --- Hàm Crawl Trang Danh sách (Phiên bản V2 - Fix Pagination) ---
def crawl_product_list_page_v2(page_url):
    """Hàm crawl một trang danh sách sản phẩm, trả về list sản phẩm và tổng số trang (nếu tìm thấy)."""
    products_on_page = []
    total_pages_output = None # This will be the value returned by the function for total pages

    try:
        # print(f"Đang crawl list page (v2): {page_url}")
        response = session.get(page_url, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml') # Sử dụng lxml cho parser mạnh mẽ hơn

        # --- Determine total_pages (robustly) ---
        # This logic should ideally run before checking for product_boxes,
        # as total_pages_output is useful even if the current page has no products (e.g. last page of results)
        pagination_el = soup.select_one('ul.pagination')

        if pagination_el:
            # Priority 1: Extract from "Trang X/Y" text if available
            page_info_element = pagination_el.select_one('li.totalPage span, li.totalPage em')
            if page_info_element:
                page_info_text = page_info_element.text.strip()
                match_pages_info = re.search(r'Trang\s*\d+\s*/\s*(\d+)', page_info_text, re.IGNORECASE)
                if match_pages_info:
                    total_pages_output = safe_int_convert(match_pages_info.group(1))
                    # if total_pages_output:
                    # print(f"DEBUG P1 [Page {page_url.split('page=')[-1]}]: Total pages from 'Trang X/Y': {total_pages_output}")


            # Priority 2: If not found via P1, try the "Cuối" link
            if total_pages_output is None:
                cuoi_link = pagination_el.select_one('li:not(.disabled) a[href*="page="]:-soup-contains("Cuối")')
                if not cuoi_link:
                     cuoi_link = pagination_el.select_one('li a[href*="page="]:-soup-contains("Cuối")')

                if cuoi_link:
                    href_cuoi = cuoi_link.get('href')
                    if href_cuoi:
                        match_cuoi = re.search(r'page=(\d+)', href_cuoi)
                        if match_cuoi:
                            total_pages_output = safe_int_convert(match_cuoi.group(1))
                            # if total_pages_output:
                            # print(f"DEBUG P2 [Page {page_url.split('page=')[-1]}]: Total pages from 'Cuối' link: {total_pages_output}")

            # Priority 3: Fallback to the maximum page number found in any pagination link
            if total_pages_output is None:
                all_page_links = pagination_el.select('li a[href*="page="]')
                candidate_page_numbers = []
                if all_page_links:
                    for link_el in all_page_links:
                        href_link = link_el.get('href')
                        if href_link:
                            match_page_num_in_href = re.search(r'page=(\d+)', href_link)
                            if match_page_num_in_href:
                                page_num = safe_int_convert(match_page_num_in_href.group(1))
                                if page_num is not None:
                                    candidate_page_numbers.append(page_num)
                    if candidate_page_numbers:
                        total_pages_output = max(candidate_page_numbers)
                        # if total_pages_output:
                        # print(f"DEBUG P3 [Page {page_url.split('page=')[-1]}]: Total pages from max page num in links: {total_pages_output}")
        # else:
            # print(f"DEBUG [Page {page_url.split('page=')[-1]}]: No 'ul.pagination' element found.")
        # --- End Determine total_pages ---


        product_boxes = soup.select('div.item_product_home')

        if not product_boxes:
            # print(f"   - Trang {page_url} không có sản phẩm.")
            # Even if no products, total_pages_output might have been found.
            # The main loop will handle stopping based on max_pages or empty page limits.
            return products_on_page, total_pages_output # Return determined total_pages_output

        # print(f"   - Tìm thấy {len(product_boxes)} sản phẩm.")
        for i, box in enumerate(product_boxes):
            product_info = {"product_id": None, "name": "", "full_name": "", "price": 0, "image_url": "", "product_url": "", "short_description": "", "ocop_rating_from_list": None}
            link = box.select_one('div.img_product a')
            href = link.get('href') if link else None
            if href:
                product_info["product_url"] = urljoin(BASE_HOST, href)
                match_id = re.search(r'goods_id=(\d+)', href)
                if match_id: product_info["product_id"] = safe_int_convert(match_id.group(1))
            else:
                # print(f"   - Box {i+1} thiếu link sản phẩm.")
                continue

            content_link = box.select_one('div.content_product a')
            if content_link:
                product_info["full_name"] = content_link.text.strip()
                short_desc = content_link.get('alt') or content_link.get('title') or ""
                product_info["short_description"] = short_desc.strip()
                match_rating = re.search(r'(\d)\s*sao\s*(?:OCOP|$)|OCOP\s*(\d)\s*sao', short_desc, re.IGNORECASE)
                if match_rating:
                    rating_str = match_rating.group(1) or match_rating.group(2)
                    product_info["ocop_rating_from_list"] = safe_int_convert(rating_str)

            product_info["name"] = re.sub(r'^✓?\s*OCOP(?:\s*\d?\s*sao)?\s*[:\s-]*', '', product_info["full_name"], flags=re.IGNORECASE).strip()
            if not product_info["name"]: product_info["name"] = product_info["full_name"]

            price_el = box.select_one('div.price_product')
            product_info["price"] = clean_price(price_el.text) if price_el else 0

            img_el = box.select_one('div.img_product img')
            img_src = img_el.get('data-src') or img_el.get('src') if img_el else None
            if img_src: product_info["image_url"] = urljoin(BASE_HOST, img_src)

            if product_info["product_id"] is not None:
                products_on_page.append(product_info)
            else:
                print(f"   - Cảnh báo: Không lấy được ID cho sản phẩm: {product_info['full_name']} tại URL: {product_info['product_url']}")

        return products_on_page, total_pages_output # Return products and determined total_pages_output

    except requests.exceptions.RequestException as e:
        print(f"   - Lỗi mạng/HTTP khi crawl list page {page_url}: {e}")
        return [], None # Trả về list rỗng và None nếu lỗi mạng (total_pages_output will be None)
    except Exception as e:
        print(f"   - Lỗi khác khi crawl list page {page_url}: {type(e).__name__} - {e}")
        # traceback.print_exc()
        return [], None # Trả về list rỗng (total_pages_output will be None)


# --- Hàm điều phối Crawl V2 (Parallel) ---
def run_crawl_v2_parallel(start_page=1):
    """Hàm điều phối việc crawl, trước tiên lấy thông tin cơ bản tuần tự, sau đó crawl chi tiết song song."""
    print("\n=== BẮT ĐẦU CRAWL DỮ LIỆU (V2 Logic - PARALLEL) ===")
    all_products_basic_info = [] # List chứa dict thông tin cơ bản
    crawled_product_ids = set() # Set để tránh crawl trùng ID
    current_page = start_page
    max_pages = None # Sẽ được cập nhật khi crawl trang đầu tiên
    consecutive_empty_pages = 0

    print("\n--- Giai đoạn 1: Thu thập thông tin cơ bản (tuần tự) ---")
    while True:
        # Điều kiện dừng
        if max_pages and current_page > max_pages:
            print(f"Đã crawl hết {max_pages} trang đã xác định.")
            break
        if current_page > CRAWL_MAX_PAGES: # CRAWL_MAX_PAGES is an overall limit
            print(f"Đạt giới hạn {CRAWL_MAX_PAGES} trang crawl (CRAWL_MAX_PAGES).")
            break

        page_url = BASE_URL_TEMPLATE.format(page=current_page)
        products_on_page, total_pages_found_on_this_page = crawl_product_list_page_v2(page_url)

        # Cập nhật tổng số trang nếu tìm thấy lần đầu VÀ nó hợp lệ ( > 0)
        if total_pages_found_on_this_page is not None and total_pages_found_on_this_page > 0 and max_pages is None:
             max_pages = total_pages_found_on_this_page
             print(f"==> Tổng số trang dự kiến được xác định: {max_pages}")
             # Adjust CRAWL_MAX_PAGES if it's set higher than the actual max_pages
             # if CRAWL_MAX_PAGES > max_pages:
             # CRAWL_MAX_PAGES = max_pages
             # print(f"    (Điều chỉnh CRAWL_MAX_PAGES thành {max_pages} để khớp với tổng trang tìm thấy)")


        # Xử lý trang trống
        if not products_on_page:
            # Kiểm tra lại xem có phải đã đến trang cuối dự kiến không
            if max_pages and current_page >= max_pages:
                print(f"Trang {current_page} trống và là trang cuối dự kiến hoặc đã vượt qua. Dừng giai đoạn 1.")
                break
            consecutive_empty_pages += 1
            print(f"Trang {current_page} trống (lần {consecutive_empty_pages}/{CRAWL_EMPTY_STOP}).")
            if consecutive_empty_pages >= CRAWL_EMPTY_STOP:
                print(f"Dừng giai đoạn 1 do {CRAWL_EMPTY_STOP} trang trống liên tiếp.")
                break
        else:
            # Reset bộ đếm trang trống và thêm sản phẩm mới
            consecutive_empty_pages = 0
            new_products_count = 0
            for product in products_on_page:
                pid = product.get('product_id')
                # Chỉ thêm nếu có ID và ID chưa được crawl
                if pid is not None and pid not in crawled_product_ids:
                    all_products_basic_info.append(product)
                    crawled_product_ids.add(pid)
                    new_products_count += 1
            if new_products_count > 0:
                 print(f"-> Trang {current_page}/{max_pages if max_pages else '?'}: Thêm {new_products_count} SP mới. Tổng cơ bản: {len(all_products_basic_info)}")
            else:
                 print(f"-> Trang {current_page}/{max_pages if max_pages else '?'}: Không có SP mới (có thể là trùng lặp ID hoặc đã được crawl).")


        current_page += 1
        time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX)) # Sleep giữa các trang list

    # --- Giai đoạn 2: Crawl chi tiết song song ---
    if not all_products_basic_info:
        print("Không có thông tin sản phẩm cơ bản nào được thu thập. Không thể crawl chi tiết.")
        return None

    print(f"\n--- Giai đoạn 2: Thu thập chi tiết ({len(all_products_basic_info)} sản phẩm) (PARALLEL với max {MAX_CRAWL_WORKERS} workers) ---")
    all_products_detailed_list = [] # List chứa dict thông tin đầy đủ
    total_to_crawl = len(all_products_basic_info)
    completed_count = 0

    with ThreadPoolExecutor(max_workers=MAX_CRAWL_WORKERS) as executor:
        futures = {executor.submit(get_product_details_v2, basic_info.get('product_url'), basic_info): basic_info
                   for basic_info in all_products_basic_info if basic_info.get('product_url')}

        for future in as_completed(futures):
            basic_info = futures[future]
            try:
                detailed_info = future.result()
                final_product_info = basic_info.copy()
                final_product_info.update(detailed_info)
                all_products_detailed_list.append(final_product_info)

            except Exception as exc:
                print(f"!!! Lỗi xảy ra khi xử lý future cho ID {basic_info.get('product_id')}: {exc}")
                error_info = basic_info.copy()
                error_info['description'] = f"Lỗi crawl parallel: {exc}"
                error_info['origin'] = error_info.get('origin', 'Lỗi crawl')
                error_info['producer'] = error_info.get('producer', 'Lỗi crawl')
                error_info['ocop_rating'] = error_info.get('ocop_rating', None) # pd.NA might be better for numeric later
                all_products_detailed_list.append(error_info)

            completed_count += 1
            if completed_count % 50 == 0 or completed_count == total_to_crawl:
                 print(f"   Đã hoàn thành crawl chi tiết {completed_count}/{total_to_crawl} sản phẩm...")

    print(f"\n--- Hoàn tất crawl parallel ---")
    if not all_products_detailed_list:
        print("Không crawl được thông tin chi tiết sản phẩm nào.")
        return None

    detailed_map = {int(info['product_id']): info for info in all_products_detailed_list if info.get('product_id') is not None}
    ordered_detailed_list = []
    for basic in all_products_basic_info:
        pid = basic.get('product_id')
        if pid is not None:
            detail = detailed_map.get(int(pid))
            if detail:
                ordered_detailed_list.append(detail)
            # else:
                # print(f"   - Cảnh báo: Không tìm thấy thông tin chi tiết đã crawl cho ID {pid}")

    if not ordered_detailed_list:
        print("Không có sản phẩm nào sau khi sắp xếp và lọc.")
        return None

    df_crawled = pd.DataFrame(ordered_detailed_list)
    final_columns_v2 = ["product_id", "name", "full_name", "price", "ocop_rating", "origin", "producer", "short_description", "description", "image_url", "product_url", "ocop_rating_from_list"]
    for col in final_columns_v2:
        if col not in df_crawled.columns:
             if col == 'price': df_crawled[col] = 0
             elif col in ['ocop_rating', 'ocop_rating_from_list']: df_crawled[col] = pd.NA
             else: df_crawled[col] = ''
    df_final = df_crawled[final_columns_v2].copy()
    return df_final

print("Block 2: Hoàn tất.")

In [ ]:
print("Block 3: Tải/Crawl Dữ liệu Chính và Làm sạch Ban đầu - Đang chạy...")

# --- Hàm Tải/Crawl và Làm sạch ---
def load_or_crawl_data(csv_path, use_parallel_crawl=True):
    """Tải dữ liệu từ CSV nếu có, ngược lại thì crawl. Trả về DataFrame và trạng thái crawl."""
    df = None
    csv_existed_before = os.path.exists(csv_path) # Kiểm tra file tồn tại trước khi thử load

    if csv_existed_before:
        print(f"Tìm thấy file CSV: {csv_path}. Đang tải...")
        try:
            df = pd.read_csv(csv_path)
            print(f"Tải thành công {len(df)} sản phẩm từ CSV.")
        except pd.errors.EmptyDataError:
            print(f"Lỗi: File CSV '{csv_path}' rỗng. Sẽ crawl lại.")
            df = None
            csv_existed_before = False # Coi như file không tồn tại để rebuild model
        except Exception as e:
            print(f"Lỗi đọc CSV: {e}. Sẽ crawl lại.")
            df = None
            csv_existed_before = False
    else:
        print(f"Không tìm thấy file CSV: {csv_path}. Sẽ crawl dữ liệu mới.")

    crawled_this_run = False # Flag để biết có crawl trong lần chạy này không
    if df is None or df.empty: # Nếu không load được hoặc file rỗng
        if use_parallel_crawl:
             print("Tiến hành crawl dữ liệu mới (V2 - Parallel)...")
             df = run_crawl_v2_parallel() # Gọi hàm crawl song song
        else:
             print("Chế độ crawl tuần tự chưa được triển khai đầy đủ trong script này.")
             # df = run_crawl_v2() # Cần định nghĩa hàm run_crawl_v2() nếu muốn dùng
             return None, False # Trả về None và chưa crawl

        crawled_this_run = True # Đánh dấu là đã crawl
        if df is not None and not df.empty:
            try:
                # Lưu dữ liệu mới crawl vào CSV
                df.to_csv(csv_path, index=False, encoding='utf-8-sig')
                print(f"Đã lưu dữ liệu crawl mới vào: {csv_path}")
            except Exception as e:
                print(f"Lỗi khi lưu file CSV mới: {e}.")
        else:
            print("Lỗi: Crawl dữ liệu không thành công hoặc không trả về DataFrame.")
            return None, crawled_this_run # Trả về None nhưng vẫn đánh dấu đã thử crawl

    # --- Làm sạch và Chuẩn hóa (Thực hiện trên df đã tải hoặc mới crawl) ---
    if df is not None and not df.empty:
        print("Đang làm sạch và chuẩn hóa dữ liệu...")
        original_row_count = len(df)
        if 'product_id' not in df.columns:
            print("LỖI nghiêm trọng: Thiếu cột 'product_id' trong DataFrame.")
            return None, crawled_this_run

        # 1. Chuyển đổi product_id sang numeric, xử lý lỗi, loại bỏ NaN/invalid
        df['product_id'] = pd.to_numeric(df['product_id'], errors='coerce')
        df_cleaned = df.dropna(subset=['product_id']).copy() # Loại bỏ hàng có product_id là NaN và tạo bản sao
        if df_cleaned.empty:
            print("Lỗi: Không có product_id hợp lệ sau khi loại bỏ NaN.")
            return None, crawled_this_run
        df_cleaned['product_id'] = df_cleaned['product_id'].astype(int) # Chuyển sang int

        # 2. Loại bỏ ID nhiễu cụ thể
        ids_to_remove = [20977, 20978] # ID của Rựa/Cuốc
        initial_len_before_remove = len(df_cleaned)
        df_filtered = df_cleaned[~df_cleaned['product_id'].isin(ids_to_remove)].copy() # Lọc và tạo bản sao
        removed_count = initial_len_before_remove - len(df_filtered)
        if removed_count > 0:
            print(f"Đã loại bỏ {removed_count} sản phẩm có ID nhiễu.")

        df_final = df_filtered
        if df_final.empty:
            print("LỖI: Không còn sản phẩm nào sau khi loại bỏ ID nhiễu.")
            return None, crawled_this_run

        # 3. Chuẩn hóa kiểu dữ liệu các cột khác và xử lý NaN
        num_cols = ['price', 'ocop_rating', 'ocop_rating_from_list']
        for col in num_cols:
             if col in df_final.columns:
                 df_final[col] = pd.to_numeric(df_final[col], errors='coerce') # Chuyển sang numeric, lỗi thành NaN (pd.NA)

        # Xử lý NaN cụ thể cho từng cột numeric
        if 'price' in df_final.columns:
            df_final['price'] = df_final['price'].fillna(0).astype(int) # Fill giá NaN = 0
        if 'ocop_rating' in df_final.columns:
            df_final['ocop_rating'] = df_final['ocop_rating'] # Giữ NaN (pd.NA) cho rating
        if 'ocop_rating_from_list' in df_final.columns:
             df_final['ocop_rating_from_list'] = df_final['ocop_rating_from_list'] # Giữ NaN (pd.NA)

        text_cols = ['name', 'full_name', 'origin', 'producer', 'short_description', 'description', 'image_url', 'product_url']
        for col in text_cols:
             if col in df_final.columns:
                 # Chuyển sang string, fill NaN bằng chuỗi rỗng
                 df_final[col] = df_final[col].astype(str).fillna('')

        # 4. Loại bỏ trùng lặp dựa trên product_id, giữ bản ghi đầu tiên
        initial_count_before_dedup = len(df_final)
        df_final = df_final.drop_duplicates(subset=['product_id'], keep='first')
        duplicates_removed = initial_count_before_dedup - len(df_final)
        if duplicates_removed > 0:
            print(f"Đã loại bỏ {duplicates_removed} bản ghi trùng lặp dựa trên product_id.")

        # 5. Reset index sau khi lọc và loại bỏ trùng lặp
        df_final = df_final.reset_index(drop=True)

        print(f"Làm sạch hoàn tất. Từ {original_row_count} ban đầu còn lại {len(df_final)} sản phẩm hợp lệ.")
        # Trả về DataFrame đã xử lý và trạng thái crawl
        return df_final, crawled_this_run
    else:
        # Trường hợp df là None hoặc empty ngay từ đầu
        print("DataFrame đầu vào không hợp lệ hoặc rỗng.")
        return None, crawled_this_run


# --- Thực thi Tải/Crawl ---
# Đặt use_parallel_crawl=True để dùng crawl song song (khuyến nghị)
df_processed, data_was_crawled = load_or_crawl_data(CSV_FILENAME, use_parallel_crawl=True)

# Kiểm tra kết quả
if df_processed is not None and not df_processed.empty:
    print("\nThông tin DataFrame đã xử lý:")
    df_processed.info()
    # print("\n5 dòng đầu:")
    # print(df_processed.head()) # Uncomment để xem thử dữ liệu
else:
    print("\nKhông thể tải hoặc crawl dữ liệu. Các bước tiếp theo có thể sẽ lỗi.")

print(f"Trạng thái crawl lần này: {data_was_crawled}")
print("Block 3: Hoàn tất.")

In [ ]:
print("Block 4: Định nghĩa Hàm Tạo Feature và Xây dựng Mô hình Nâng cao - Đang chạy...")

# --- Hàm Tạo Feature Tổng hợp (Phiên bản V2) ---
def combine_features_v2(row):
    """Kết hợp các trường text thành một chuỗi duy nhất để tính TF-IDF."""
    # Lấy giá trị từ các cột, đảm bảo là string, xử lý NaN/None
    name_part = str(row.get('name', '')) * NAME_WEIGHT # Nhân trọng số cho tên
    origin_part = str(row.get('origin', ''))
    producer_part = str(row.get('producer', ''))
    description_part = str(row.get('description', ''))
    short_desc_part = str(row.get('short_description', ''))

    # Tạo list các phần text, chỉ bao gồm các chuỗi không rỗng và không phải 'nan'
    feature_parts = [part for part in [name_part, origin_part, producer_part, short_desc_part, description_part]
                     if isinstance(part, str) and part.strip() and part.strip().lower() != 'nan']

    # Kết hợp các phần thành một chuỗi lớn
    combined_text = " ".join(feature_parts)

    # Áp dụng từ đồng nghĩa trên chuỗi kết hợp
    return apply_synonyms(combined_text)

# --- Hàm Xây dựng Mô hình TF-IDF (Sử dụng tokenizer nâng cao và tham số tối ưu) ---
def build_tfidf_model_advanced(df):
    """Xây dựng ma trận TF-IDF và Cosine Similarity từ DataFrame sản phẩm."""
    if df is None or df.empty or 'product_id' not in df.columns:
        print("Lỗi: DataFrame không hợp lệ để xây dựng mô hình.")
        return None, None, None

    print("\n--- Bắt đầu Xây dựng mô hình TF-IDF Nâng cao ---")

    df_model = df.copy()
    # 1. Chuẩn hóa product_id: Chuyển sang numeric để loại bỏ NaN, sau đó chuyển sang STRING
    df_model['product_id'] = pd.to_numeric(df_model['product_id'], errors='coerce')
    df_model = df_model.dropna(subset=['product_id'])
    if df_model.empty:
        print("LỖI: Không có product_id hợp lệ sau khi loại bỏ NaN.")
        return None, None, None

    # >>> SỬA Ở ĐÂY: Chuyển product_id thành STRING <<<
    df_model['product_id'] = df_model['product_id'].astype(int).astype(str) # Chuyển sang int rồi mới sang str để loại bỏ '.0' nếu có

    df_model = df_model.reset_index(drop=True)

    # ... (phần tạo combined_features, TF-IDF, Cosine Similarity giữ nguyên) ...
    print("Đang tạo cột 'combined_features' (v2)...")
    df_model['combined_features'] = df_model.apply(combine_features_v2, axis=1)
    print("Tạo 'combined_features' hoàn tất.")

    print("Đang khởi tạo TfidfVectorizer Nâng cao...")
    tfidf_vectorizer = TfidfVectorizer(
        tokenizer=advanced_tokenizer,
        ngram_range=TFIDF_NGRAM_RANGE,
        min_df=TFIDF_MIN_DF,
        max_df=TFIDF_MAX_DF,
        max_features=TFIDF_MAX_FEATURES,
        sublinear_tf=TFIDF_SUBLINEAR_TF,
        norm=TFIDF_NORM
    )
    print("Đang tính toán ma trận TF-IDF...")
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_model['combined_features'])
    print(f"Kích thước ma trận TF-IDF: {tfidf_matrix.shape}")

    if tfidf_matrix.shape[0] == 0 or tfidf_matrix.shape[1] == 0:
        print("LỖI: Ma trận TF-IDF rỗng!")
        return None, None, df_model

    print("Đang tính toán ma trận Cosine Similarity...")
    cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    print(f"Kích thước ma trận Cosine Similarity: {cosine_sim_matrix.shape}")

    # 5. Tạo Map từ product_id (STRING) sang index của DataFrame (df_model)
    # Index của Series bây giờ sẽ là product_id dạng STRING
    indices = pd.Series(df_model.index, index=df_model['product_id']).drop_duplicates()
    print(f"Tạo ánh xạ product_id (string) sang index hoàn tất. Số lượng ánh xạ: {len(indices)}")
    # Kiểm tra kiểu dữ liệu của index của Series 'indices'
    # print(f"DEBUG: Kiểu dữ liệu của index trong 'indices' Series: {indices.index.dtype}")


    return cosine_sim_matrix, indices, df_model

print("Block 4: Hoàn tất.")

In [ ]:
print("Block 5: Thực thi Xây dựng Mô hình Nâng cao và Lưu trữ JSON - Đang chạy...")

cosine_sim_matrix = None
indices = None # Đây sẽ là Pandas Series với index là product_id (string)
df_for_map_creation = None

# Chỉ thực hiện nếu df_processed từ Block 3 hợp lệ và không rỗng
if 'df_processed' in locals() and isinstance(df_processed, pd.DataFrame) and not df_processed.empty:

    should_rebuild_model = True

    # Kiểm tra xem có cần xây dựng lại model không
    if not data_was_crawled and os.path.exists(COSINE_SIM_MATRIX_FILE) and os.path.exists(INDICES_MAP_FILE):
        print("Kiểm tra file ma trận và indices đã lưu (do không crawl lần này)...")
        try:
            temp_cosine_matrix = np.load(COSINE_SIM_MATRIX_FILE)
            with open(INDICES_MAP_FILE, 'rb') as f:
                temp_indices = pickle.load(f) # temp_indices là Pandas Series, index là product_id (string)

            # Đảm bảo temp_indices là Series và index của nó là string
            if not isinstance(temp_indices, pd.Series):
                print("Lỗi: File indices đã lưu không phải là Pandas Series. Sẽ xây dựng lại.")
                raise ValueError("Saved indices is not a Pandas Series.")
            temp_indices.index = temp_indices.index.astype(str) # Đảm bảo index là string


            # Chuẩn hóa product_id trong df_processed để so sánh
            # Chuyển sang numeric để bỏ NaN, sau đó int, rồi string
            df_processed_check = df_processed.copy()
            df_processed_check['product_id_str_check'] = pd.to_numeric(df_processed_check['product_id'], errors='coerce')
            df_processed_check.dropna(subset=['product_id_str_check'], inplace=True)

            valid_df_count = 0
            if not df_processed_check.empty:
                 df_processed_check['product_id_str_check'] = df_processed_check['product_id_str_check'].astype(int).astype(str)
                 valid_df_count = len(df_processed_check['product_id_str_check'].unique())

            # So sánh số lượng product_id duy nhất
            # temp_indices.index chứa các product_id (string)
            if len(temp_indices.index.unique()) == valid_df_count and \
               temp_cosine_matrix.shape[0] == len(temp_indices.index.unique()):
                print(f"Kích thước khớp ({valid_df_count}). Sử dụng ma trận/indices đã lưu.")
                cosine_sim_matrix = temp_cosine_matrix
                indices = temp_indices # indices này có index là product_id (string)
                should_rebuild_model = False
                df_for_map_creation = df_processed.copy()
            else:
                print(f"Kích thước không khớp (Indices đã lưu: {len(temp_indices.index.unique()) if isinstance(temp_indices, pd.Series) else 'N/A'}, Matrix: {temp_cosine_matrix.shape[0]}, DataFrame hợp lệ: {valid_df_count}). Sẽ xây dựng lại mô hình...")
        except Exception as e:
            print(f"Lỗi khi tải hoặc kiểm tra file ma trận/indices đã lưu: {e}. Sẽ xây dựng lại mô hình...")
            traceback.print_exc() # In traceback để debug
    elif data_was_crawled:
        print("Dữ liệu mới đã được crawl trong lần chạy này. Sẽ xây dựng lại mô hình.")
    else:
         print("Không có file ma trận/indices đã lưu và không crawl mới. Sẽ xây dựng mô hình.")

    if should_rebuild_model:
        # df_processed ở đây là DataFrame gốc từ Block 3
        # Hàm build_tfidf_model_advanced sẽ xử lý việc chuyển product_id sang string bên trong nó
        temp_cosine_sim_matrix, temp_indices, df_model_updated = build_tfidf_model_advanced(df_processed)

        if temp_cosine_sim_matrix is not None and temp_indices is not None and df_model_updated is not None:
            print("Xây dựng mô hình thành công.")
            cosine_sim_matrix = temp_cosine_sim_matrix
            indices = temp_indices # indices này có index là product_id (string)
            df_for_map_creation = df_model_updated.copy() # Dùng df đã được xử lý bởi build_tfidf_model_advanced

            try:
                print("Đang lưu trữ ma trận cosine và map indices mới...")
                np.save(COSINE_SIM_MATRIX_FILE, cosine_sim_matrix)
                with open(INDICES_MAP_FILE, 'wb') as f:
                    pickle.dump(indices, f) # Lưu Series indices
                print(f"Đã lưu thành công: {COSINE_SIM_MATRIX_FILE}, {INDICES_MAP_FILE}")
            except Exception as e:
                print(f"LỖI nghiêm trọng khi lưu file ma trận/indices: {e}")
                traceback.print_exc()
                cosine_sim_matrix = None # Reset nếu không lưu được
                indices = None
        else:
            print("LỖI: Không thể xây dựng mô hình TF-IDF/Cosine Similarity. Bỏ qua các bước sau.")
            cosine_sim_matrix = None
            indices = None
            df_for_map_creation = None # Không có df để tạo map

    # --- Tạo và Lưu Map Sản phẩm (PRODUCT_MAP_JSON_FILE) ---
    if df_for_map_creation is not None and not df_for_map_creation.empty and 'product_id' in df_for_map_creation.columns:
        print("\nĐang tạo và lưu Map thông tin sản phẩm (JSON)...")
        try:
            map_columns = ['product_id', 'name', 'origin', 'producer', 'image_url',
                           'price', 'ocop_rating', 'product_url', 'full_name',
                           'category', 'short_description', 'description', 'num_reviews', 'sold'] # Thêm các trường cần thiết

            existing_map_columns = [col for col in map_columns if col in df_for_map_creation.columns]
            df_map = df_for_map_creation[existing_map_columns].copy()

            # 1. Chuẩn hóa product_id sang string để làm key JSON (đã được làm trong build_tfidf_model_advanced nếu model được build lại)
            # Nếu model được load từ file, df_for_map_creation là df_processed, cần chuẩn hóa ở đây
            if 'product_id' in df_map.columns: # Kiểm tra lại
                df_map['product_id_numeric'] = pd.to_numeric(df_map['product_id'], errors='coerce')
                df_map.dropna(subset=['product_id_numeric'], inplace=True)
                if not df_map.empty:
                    df_map['product_id'] = df_map['product_id_numeric'].astype(int).astype(str)
                df_map.drop(columns=['product_id_numeric'], inplace=True, errors='ignore')


            # 2. Xử lý NaN/NA và đảm bảo kiểu dữ liệu đúng cho JSON
            for col in ['name', 'origin', 'producer', 'image_url', 'product_url', 'full_name', 'category', 'short_description', 'description']:
                if col in df_map.columns:
                    df_map[col] = df_map[col].fillna('').astype(str)

            for col in ['price', 'ocop_rating', 'num_reviews', 'sold']: # Thêm num_reviews, sold
                if col in df_map.columns:
                    df_map[col] = pd.to_numeric(df_map[col], errors='coerce') # Chuyển sang số
                    # Đối với rating, num_reviews, sold có thể muốn fillna bằng 0 hoặc giữ NA rồi xử lý khi dump JSON
                    if col == 'price':
                        df_map[col] = df_map[col].fillna(0).astype(int)
                    elif col == 'ocop_rating':
                         # Chuyển float NaN thành None, giữ nguyên số nguyên/float khác
                        df_map[col] = df_map[col].apply(lambda x: safe_int_convert(x) if pd.notnull(x) else None)
                    elif col in ['num_reviews', 'sold']:
                         df_map[col] = df_map[col].fillna(0).astype(int)


            # 3. Tạo dictionary với product_id (string) làm key
            if df_map.empty or 'product_id' not in df_map.columns or df_map['product_id'].isnull().all():
                print("Lỗi: Không có product_id hợp lệ trong df_map để tạo product_map_dict.")
                product_map_dict = {}
            else:
                # Loại bỏ các hàng có product_id là NaN/None một lần nữa trước khi set_index
                df_map.dropna(subset=['product_id'], inplace=True)
                # Đảm bảo product_id là duy nhất trước khi set_index, giữ bản ghi đầu tiên nếu trùng
                df_map.drop_duplicates(subset=['product_id'], keep='first', inplace=True)
                if not df_map.empty:
                    product_map_dict = df_map.set_index('product_id').to_dict('index')
                else:
                    product_map_dict = {}


            # 4. Lưu dictionary vào file JSON
            with open(PRODUCT_MAP_JSON_FILE, 'w', encoding='utf-8') as f:
                 json.dump(product_map_dict, f, ensure_ascii=False, indent=2)

            print(f"Đã lưu map sản phẩm vào: {PRODUCT_MAP_JSON_FILE} với {len(product_map_dict)} sản phẩm.")
        except Exception as e:
            print(f"Lỗi khi tạo hoặc lưu file map sản phẩm JSON: {e}")
            traceback.print_exc()
    else:
        print("Không có DataFrame hợp lệ ('df_for_map_creation') để tạo map sản phẩm.")

else:
    print("Không có dữ liệu 'df_processed' từ Block 3 để thực hiện Block 5.")

print("Block 5: Hoàn tất.")

In [ ]:
print("Block 6: Định nghĩa Hàm Gợi ý và Tiền tính toán - Đang chạy...")

# --- Hàm Gợi ý ID Sản phẩm (Dùng cho tiền tính toán) ---
def get_recommendation_ids_for_precomputation(product_id_int, indices_map, cosine_matrix, index_to_id_map, top_n=30):
    """Lấy list các ID sản phẩm gợi ý (dạng int) cho một ID đầu vào."""
    if product_id_int not in indices_map:
        # print(f"  Debug: ID {product_id_int} không có trong indices_map.")
        return [] # Trả về list rỗng nếu ID không có trong map

    idx = indices_map[product_id_int] # Lấy DataFrame index từ product_id

    # Kiểm tra index hợp lệ so với kích thước ma trận
    if idx >= cosine_matrix.shape[0]:
        # print(f"  Debug: Chỉ số DataFrame ({idx}) cho ID {product_id_int} vượt quá kích thước ma trận ({cosine_matrix.shape[0]}).")
        return []

    try:
        # Lấy hàng tương ứng trong ma trận cosine, kèm theo index gốc
        sim_scores_with_indices = list(enumerate(cosine_matrix[idx]))

        # Sắp xếp theo điểm số giảm dần
        sim_scores_with_indices = sorted(sim_scores_with_indices, key=lambda x: x[1], reverse=True)

        # Lấy top_n+1 gợi ý (bao gồm cả chính nó) và bỏ đi gợi ý đầu tiên (chính nó)
        top_recommendations_indices = [score_tuple[0] for score_tuple in sim_scores_with_indices[1 : top_n + 1]]

        # Chuyển đổi DataFrame indices thành product_ids (int) dùng map ngược
        recommended_ids_int = [index_to_id_map.get(rec_idx) for rec_idx in top_recommendations_indices]
        # Lọc bỏ các giá trị None (nếu có lỗi trong map ngược)
        recommended_ids_int = [pid for pid in recommended_ids_int if pid is not None]

        return recommended_ids_int # Trả về list các product ID (integer)

    except IndexError:
         print(f"Lỗi IndexError khi truy cập cosine_matrix[{idx}] cho ID {product_id_int}.")
         return []
    except Exception as e:
        print(f"Lỗi không xác định khi lấy gợi ý cho ID {product_id_int}: {e}")
        # traceback.print_exc() # Uncomment để debug
        return []


# --- Hàm Tiền tính toán Gợi ý (Thô - JSON) ---
def precompute_raw_recommendations_json(all_product_ids_int, indices_map, cosine_matrix, top_n=30):
    """Tiền tính toán và trả về dict {"product_id_str": [rec_id_int_1, rec_id_int_2,...]}."""
    precomputed_recs_dict = {} # Dict để lưu kết quả
    total_products = len(all_product_ids_int)
    print(f"\n--- Bắt đầu Tiền tính toán Top {top_n} gợi ý THÔ cho {total_products} sản phẩm ---")

    # Tạo map ngược từ DataFrame index -> product_id (int) để tra cứu nhanh
    try:
        index_to_id_map = {v: k for k, v in indices_map.items()}
        print(f"Đã tạo map ngược index -> product_id thành công ({len(index_to_id_map)} mục).")
    except Exception as e:
        print(f"Lỗi khi tạo map ngược index->ID: {e}. Việc tra cứu ID gợi ý có thể chậm hơn.")
        index_to_id_map = None # Sẽ phải duyệt indices_map nếu không tạo được map ngược

    processed_count = 0
    # Duyệt qua từng product ID cần tính toán
    for prod_id_int in all_product_ids_int:
        # Gọi hàm lấy list ID gợi ý cho ID hiện tại
        recommended_ids = get_recommendation_ids_for_precomputation(
            prod_id_int,
            indices_map,
            cosine_matrix,
            index_to_id_map, # Truyền map ngược vào
            top_n=top_n
        )

        # Lưu kết quả vào dict, với key là product ID dạng string
        precomputed_recs_dict[str(prod_id_int)] = recommended_ids

        processed_count += 1
        # In tiến trình
        if processed_count % 200 == 0 or processed_count == total_products:
            print(f"  Đã xử lý tiền tính toán cho {processed_count}/{total_products} sản phẩm...")

    print(f"--- Tiền tính toán gợi ý thô hoàn tất ({len(precomputed_recs_dict)} sản phẩm có gợi ý) ---")
    return precomputed_recs_dict

# --- Hàm Lấy Gợi ý Chi tiết (Dùng cho kiểm tra cuối cùng) ---
def get_final_recommendations_with_details(product_id_int, precomputed_recs_dict, product_map_dict, top_n=10):
    """Lấy thông tin chi tiết cho các ID gợi ý đã được tiền tính toán."""
    product_id_str = str(product_id_int)
    recommended_ids_int = precomputed_recs_dict.get(product_id_str, [])

    if not recommended_ids_int:
        # print(f"Không có gợi ý tiền tính toán cho ID {product_id_int}.")
        return []

    recommendations_details = []
    # Lấy tối đa top_n gợi ý đầu tiên từ list đã tiền tính toán
    for rec_id_int in recommended_ids_int[:top_n]:
        rec_id_str = str(rec_id_int)
        rec_info = product_map_dict.get(rec_id_str) # Lấy thông tin từ map sản phẩm
        if rec_info:
            # Tạo một bản sao để tránh thay đổi dict gốc
            detail = rec_info.copy()
            detail['product_id'] = rec_id_int # Đảm bảo product_id là int trong kết quả cuối
            # Score không có sẵn ở đây vì chỉ lưu ID, cần tính lại nếu muốn hiển thị score
            # detail['similarity_score'] = ... # Cần tính lại nếu muốn score
            recommendations_details.append(detail)
        # else:
            # print(f"  Cảnh báo: ID gợi ý {rec_id_int} không tìm thấy trong product_map_dict.")

    return recommendations_details


print("Block 6: Hoàn tất.")

In [ ]:
print("Block 7: Thực thi Tiền tính toán và Lưu Gợi ý JSON - Đang chạy...")

precomputed_recommendations_dict = None # Dict chứa kết quả tiền tính toán

# Chỉ thực hiện nếu ma trận và indices đã được tạo/tải thành công ở Block 5
if 'indices' in locals() and indices is not None and \
   'cosine_sim_matrix' in locals() and cosine_sim_matrix is not None:

    print(f"Dữ liệu đầu vào cho tiền tính toán: Indices({len(indices)}), Matrix({cosine_sim_matrix.shape})")
    try:
        # Lấy danh sách các product ID duy nhất (dạng int) từ index của Series 'indices'
        all_product_ids_to_precompute = indices.index.unique().tolist()
        print(f"Số lượng ID sản phẩm duy nhất cần tiền tính toán: {len(all_product_ids_to_precompute)}")

        if not all_product_ids_to_precompute:
             print("LỖI: Không có ID sản phẩm nào để tiền tính toán.")
        else:
            # Gọi hàm tiền tính toán gợi ý (sử dụng TOP_N_RAW_RECS)
            precomputed_recommendations_dict = precompute_raw_recommendations_json(
                all_product_ids_to_precompute,
                indices,               # Map product_id (int) -> df_index (int)
                cosine_sim_matrix,     # Ma trận cosine similarity
                top_n=TOP_N_RAW_RECS   # Số lượng gợi ý thô cần lưu cho mỗi sản phẩm
            )

            # Lưu kết quả tiền tính toán vào file JSON
            if precomputed_recommendations_dict: # Chỉ lưu nếu có kết quả
                print(f"Đang lưu gợi ý tiền tính toán vào: {PRECOMPUTED_RECS_JSON_FILE}")
                try:
                    with open(PRECOMPUTED_RECS_JSON_FILE, 'w', encoding='utf-8') as f:
                        # Lưu dict {"product_id_str": [rec_id_int_1, rec_id_int_2,...]}
                        json.dump(precomputed_recommendations_dict, f, ensure_ascii=False, indent=2)
                    print(f"Đã lưu thành công {len(precomputed_recommendations_dict)} sản phẩm vào {PRECOMPUTED_RECS_JSON_FILE}")
                except Exception as save_err:
                    print(f"LỖI khi lưu file gợi ý JSON: {save_err}")
            else:
                 print("Không có kết quả gợi ý nào được tạo ra để lưu.")

    except Exception as e:
        print(f"Lỗi nghiêm trọng trong quá trình tiền tính toán và lưu gợi ý: {e}")
        traceback.print_exc()
else:
    print("Thiếu dữ liệu đầu vào (indices hoặc cosine_sim_matrix) từ Block 5. Không thể tiền tính toán gợi ý.")

print("Block 7: Hoàn tất.")

In [ ]:
# ==============================================================================
# --- BLOCK 8: KIỂM TRA GỢI Ý CUỐI CÙNG (SỬ DỤNG FILE JSON VÀ TÍNH LẠI SCORE) ---
# ==============================================================================
print("\nBlock 8: Kiểm tra Gợi ý Cuối cùng (Sử dụng file JSON và tính lại score) - Đang chạy...")

# --- ĐỊNH NGHĨA LẠI HOẶC ĐẢM BẢO CÁC HÀM TIỆN ÍCH ĐÃ CÓ ---
# (Nếu các hàm này đã được định nghĩa ở block trước và chạy đúng, bạn không cần định nghĩa lại)
def safe_int_convert(value):
    try: return int(float(value)) # Chuyển qua float trước để xử lý "3.0"
    except (ValueError, TypeError, OverflowError): return None

def safe_float_convert(value):
    try: return float(value)
    except (ValueError, TypeError, OverflowError): return None
# --- KẾT THÚC ĐỊNH NGHĨA HÀM TIỆN ÍCH ---


# --- HÀM LẤY GỢI Ý CHI TIẾT VÀ SCORE ---
def get_final_recommendations_with_details_and_scores(
    product_id_input_str,
    precomputed_recs_dict,
    product_map_dict,
    cosine_sim_matrix_loaded,
    indices_map_loaded_series,
    top_n=TOP_N_FINAL_RECS # TOP_N_FINAL_RECS cần được định nghĩa ở scope này hoặc global
):
    recommended_ids_int_list = precomputed_recs_dict.get(product_id_input_str)

    if not recommended_ids_int_list:
        return []

    recommendations_with_details = []

    if product_id_input_str not in indices_map_loaded_series.index:
        return []
    original_product_matrix_idx = indices_map_loaded_series[product_id_input_str]

    if not isinstance(original_product_matrix_idx, (int, np.integer)):
        return []

    for rec_id_int in recommended_ids_int_list[:top_n]:
        rec_id_str = str(rec_id_int)
        rec_product_details = product_map_dict.get(rec_id_str)

        if rec_product_details:
            score = 0.0
            if rec_id_str in indices_map_loaded_series.index:
                recommended_product_matrix_idx = indices_map_loaded_series[rec_id_str]
                if isinstance(recommended_product_matrix_idx, (int, np.integer)):
                    if (0 <= original_product_matrix_idx < cosine_sim_matrix_loaded.shape[0] and
                        0 <= recommended_product_matrix_idx < cosine_sim_matrix_loaded.shape[1]):
                        score = cosine_sim_matrix_loaded[original_product_matrix_idx, recommended_product_matrix_idx]

            recommendations_with_details.append({
                "product_id": rec_id_int,
                "name": rec_product_details.get('name', 'N/A'),
                "origin": rec_product_details.get('origin', 'N/A'),
                "producer": rec_product_details.get('producer', 'N/A'),
                "price": safe_float_convert(rec_product_details.get('price', 0)), # Đã có thể gọi
                "ocop_rating": safe_int_convert(rec_product_details.get('ocop_rating')), # Đã có thể gọi
                "image_url": rec_product_details.get('image_url', ''),
                "product_url": rec_product_details.get('product_url', ''),
                "similarity_score": float(score) if pd.notnull(score) else 0.0
            })
    return recommendations_with_details

# --- KẾT THÚC HÀM LẤY GỢI Ý ---


# Kiểm tra sự tồn tại của các file
map_exists = os.path.exists(PRODUCT_MAP_JSON_FILE)
recs_exists = os.path.exists(PRECOMPUTED_RECS_JSON_FILE)
matrix_exists = os.path.exists(COSINE_SIM_MATRIX_FILE)
indices_file_exists = os.path.exists(INDICES_MAP_FILE)

if map_exists and recs_exists and matrix_exists and indices_file_exists:
    print("\n--- KIỂM TRA GỢI Ý CUỐI CÙNG (TỪ FILE JSON, TÍNH LẠI SCORE) ---")
    product_map_loaded_dict = None
    precomputed_recs_loaded_dict = None
    cosine_sim_matrix_loaded = None
    indices_map_loaded_series = None # Đổi tên biến cho rõ đây là Series

    try:
        print(f"Đang tải map sản phẩm từ JSON: {PRODUCT_MAP_JSON_FILE}")
        with open(PRODUCT_MAP_JSON_FILE, 'r', encoding='utf-8') as f:
             product_map_loaded_dict = json.load(f)
        print(f"Tải map sản phẩm thành công ({len(product_map_loaded_dict)} sản phẩm).")

        print(f"Đang tải gợi ý tiền tính toán từ JSON: {PRECOMPUTED_RECS_JSON_FILE}")
        with open(PRECOMPUTED_RECS_JSON_FILE, 'r', encoding='utf-8') as f:
            precomputed_recs_loaded_dict = json.load(f)
        print(f"Tải gợi ý tiền tính toán thành công ({len(precomputed_recs_loaded_dict)} sản phẩm có gợi ý).")

        print(f"Đang tải ma trận cosine similarity từ: {COSINE_SIM_MATRIX_FILE}")
        cosine_sim_matrix_loaded = np.load(COSINE_SIM_MATRIX_FILE)
        print(f"Tải ma trận thành công (shape: {cosine_sim_matrix_loaded.shape}).")

        print(f"Đang tải map product ID sang index (Pandas Series) từ: {INDICES_MAP_FILE}")
        with open(INDICES_MAP_FILE, 'rb') as f:
            indices_map_loaded_series = pickle.load(f) # Load Pandas Series

        if not isinstance(indices_map_loaded_series, pd.Series):
            print(f"LỖI: File {INDICES_MAP_FILE} không chứa đối tượng Pandas Series hợp lệ.")
            indices_map_loaded_series = None # Đặt là None nếu không đúng
        elif indices_map_loaded_series.index.dtype != 'object': # Kiểm tra kiểu index, nên là object (cho string)
             # Chuyển đổi index của Series thành string nếu nó chưa phải
             indices_map_loaded_series.index = indices_map_loaded_series.index.astype(str)
             print(f"Tải map index (Pandas Series) thành công ({len(indices_map_loaded_series)} entries). Index được chuyển thành string.")
        else:
             print(f"Tải map index (Pandas Series) thành công ({len(indices_map_loaded_series)} entries). Index type: {indices_map_loaded_series.index.dtype}.")


        if product_map_loaded_dict and precomputed_recs_loaded_dict and \
           cosine_sim_matrix_loaded is not None and indices_map_loaded_series is not None and not indices_map_loaded_series.empty:

            available_ids_in_recs_str = list(precomputed_recs_loaded_dict.keys()) # Đây là các product_id_str

            if not available_ids_in_recs_str:
                print("LỖI: Không có ID nào trong file gợi ý đã tải (precomputed_recs_loaded_dict).")
            else:
                valid_ids_for_testing_str = []
                for id_str_candidate in available_ids_in_recs_str:
                    # id_str_candidate đã là string từ key của precomputed_recs_loaded_dict
                    # Kiểm tra xem id_str_candidate có trong index của indices_map_loaded_series không
                    # và có trong product_map_loaded_dict (dict) không
                    if id_str_candidate in indices_map_loaded_series.index and \
                       id_str_candidate in product_map_loaded_dict:
                        valid_ids_for_testing_str.append(id_str_candidate)

                if not valid_ids_for_testing_str:
                    print("LỖI: Không có ID nào từ precomputed_recs hợp lệ để kiểm tra (không khớp giữa precomputed_recs, indices_map và product_map).")
                else:
                    num_random_samples = min(5, len(valid_ids_for_testing_str)) # Tránh lỗi nếu valid_ids < 5
                    ids_to_test_str = random.sample(valid_ids_for_testing_str, num_random_samples)
                    print(f"Chọn ngẫu nhiên {num_random_samples} ID hợp lệ để kiểm tra: {ids_to_test_str}")

                    for test_product_id_str in ids_to_test_str:
                        print(f"\n--- Đang kiểm tra cho ID_STR: {test_product_id_str} ---")
                        original_product_info = product_map_loaded_dict.get(test_product_id_str)

                        if not original_product_info:
                            print(f"  Cảnh báo: Không tìm thấy thông tin sản phẩm gốc cho ID_STR {test_product_id_str} trong product_map.")
                            continue

                        print("="*50)
                        print(f"Sản phẩm Gốc (ID_STR: {test_product_id_str})") # In ID string
                        print(f"  Tên: {original_product_info.get('name', 'N/A')}")
                        print(f"  Xuất xứ: {original_product_info.get('origin', 'N/A')}")
                        print(f"  Nhà sản xuất: {original_product_info.get('producer', 'N/A')}")
                        print("-" * 20)
                        print(f"Gợi ý (Top {TOP_N_FINAL_RECS}):")

                        recommendations_details = get_final_recommendations_with_details_and_scores(
                            test_product_id_str, # Truyền product_id dạng string
                            precomputed_recs_loaded_dict,
                            product_map_loaded_dict,
                            cosine_sim_matrix_loaded,
                            indices_map_loaded_series,
                            top_n=TOP_N_FINAL_RECS
                        )

                        if recommendations_details:
                            print(f"  Tìm thấy {len(recommendations_details)} gợi ý chi tiết.")
                            for i, rec_detail in enumerate(recommendations_details):
                                score_display = f"{rec_detail.get('similarity_score', 0.0):.4f}"
                                print(f"  {i+1}. ID: {rec_detail.get('product_id','Lỗi ID')} (Score: {score_display})")
                                print(f"     Tên: {rec_detail.get('name', 'N/A')}")
                        else:
                            print(f"  - Không có gợi ý chi tiết nào được tìm thấy cho ID này.")
                        print("="*50)
        else:
            print("Một hoặc nhiều file artifact cần thiết không được tải hoặc rỗng.")

    except FileNotFoundError as e: print(f"\nLỖI: Không tìm thấy file cần thiết: {e}")
    except json.JSONDecodeError as e: print(f"\nLỖI: File JSON không hợp lệ: {e}")
    except pickle.UnpicklingError as e: print(f"\nLỖI: File Pickle không hợp lệ: {e}")
    except Exception as e:
        print(f"\nLỖI không xác định: {type(e).__name__} - {e}")
        traceback.print_exc()
else:
    print("\nKhông thể kiểm tra gợi ý do thiếu file:")
    if not map_exists: print(f" - Thiếu: {PRODUCT_MAP_JSON_FILE}")
    if not recs_exists: print(f" - Thiếu: {PRECOMPUTED_RECS_JSON_FILE}")
    if not matrix_exists: print(f" - Thiếu: {COSINE_SIM_MATRIX_FILE}")
    if not indices_file_exists: print(f" - Thiếu: {INDICES_MAP_FILE}")

print("\nBlock 8: Hoàn tất.")
print("\n--- Chương trình kết thúc ---")